In [1]:
#include "Math/GenVector/PxPyPzE4D.h"
#include "Math/GenVector/DisplacementVector3D.h"
#include "Math/GenVector/GenVectorIO.h"
#include "Math/GenVector/LorentzVector.h"

In [2]:
typedef ROOT::Math::LorentzVector<ROOT::Math::PxPyPzE4D<double> > XYZTLorentzVectorD;
typedef XYZTLorentzVectorD XYZTLorentzVector;

In [3]:
Double_t DeltaPhi(XYZTLorentzVector a,XYZTLorentzVector b){
    Double_t deltaphi = (a.Phi()-b.Phi());
    if (deltaphi >= 3.14159265) deltaphi -= 2*3.14159265;
    if (deltaphi < -3.14159265) deltaphi += 2*3.14159265;
    return deltaphi;
}

In [4]:
std::string FilePath = "/home/camilo/cms/VLF_Montecarlo_Ana/Data/";
//std::string FileName = "Signal.root";
//std::string FileName = "DYJetsToLL_HT/DYJetsToLL_M-50_ext1-v2.root";
//std::string FileName = "DYJetsToLL_HT/DYJetsToLL_M-50_ext2.root";

//std::string FileName = "ST/ST_t-channel_top_4f_inclusiveDecays.root";
//std::string FileName = "ST/ST_s-channel_4f_leptonDecays.root";
//std::string FileName = "ST/ST_t-channel_antitop_4f_inclusiveDecays.root";
//std::string FileName = "ST/ST_tW_antitop_5f_inclusiveDecays.root";
//std::string FileName = "ST/ST_tW_top_5f_inclusiveDecays.root";

std::string FileName = "TTToLL_MLL/TT_TuneCUETP8M2T4.root";

//std::string FileName = "WJets_HT/WJetsToLNu_HT-70To100.root";
//std::string FileName = "WJets_HT/WJetsToLNu_HT-100To200.root";
//std::string FileName = "WJets_HT/WJetsToLNu_HT-200To400.root";
//std::string FileName = "WJets_HT/WJetsToLNu_HT-400To600.root";
//std::string FileName = "WJets_HT/WJetsToLNu_HT-600To800.root";
//std::string FileName = "WJets_HT/WJetsToLNu_HT-800To1200.root";
//std::string FileName = "WJets_HT/WJetsToLNu_HT-1200To2500.root";
//std::string FileName = "WJets_HT/WJetsToLNu_HT-2500ToInf.root";

std::string File = FilePath+FileName;
std::string PathTree = "demo/VLFTree";

In [5]:
TFile *f = new TFile(File.c_str(),"update"); //Open File
TTree *Tree = (TTree*)f->Get(PathTree.c_str()); //Get the Tree
int nevent = Tree->GetEntries();
//Int_t count = 0;

In [6]:
std::vector<XYZTLorentzVector> *AllMuons = new std::vector<XYZTLorentzVector>();
Tree->SetBranchAddress("AllMuons", &AllMuons);
XYZTLorentzVector * MET = new XYZTLorentzVector();
Tree->SetBranchAddress("AnaMET", &MET);
std::vector <bool> *Muon_Medium;
Tree->SetBranchAddress("AllMuonMediumID", &Muon_Medium);
std::vector <double> *Combined_Iso;
Tree->SetBranchAddress("AllCombined_iso_DeltaBetaPU", &Combined_Iso);

In [7]:
//NEW Branches
XYZTLorentzVector  TheMuon;
TBranch *TheMuonB = Tree->Branch("TheMuon",&TheMuon); 
int Mupos = 100;
TBranch *TheMuonPossB = Tree->Branch("MuPoss",&Mupos); 
double MT_TheMuon_MET;
TBranch *MT_TheMuon_METB = Tree->Branch("MT_TheMuon_MET",&MT_TheMuon_MET); 

In [8]:
int countAll, countPass, countX2;

In [9]:
countAll=0, countPass=0, countX2=0;
for (int i = 0 ; i < nevent ; i++ ) {
    Tree->GetEvent(i);              //read complete accepted event in memory
    //std::cout<<i<<" = "<<AnaMuons->at(0).Pt()<<std::endl;
    //std::cout<<AllMuons->size()<<std::endl;
    countAll++;
    Double_t tempMT = -1000.0;
    //XYZTLorentzVector  TheMuon;
    int nmuons = AllMuons->size();
    int countX2Temp=0;
    //int Mupos= 100;//Possition in the vector
    for (int a = 0 ; a <nmuons; a++ ) {
        //std::cout<<i<<","<<a<<" = "<<AllMuons->at(a).Pt()<<std::endl;
        if (AllMuons->at(a).pt()>3.0 && AllMuons->at(a).pt() < 60.0){
            if (Muon_Medium->at(a)){
                if(Combined_Iso->at(a)<0.15){//0.15
                    Double_t MT_LeadingMuon_MET = sqrt(2*AllMuons->at(a).Pt()*MET->Pt()*(1-cos(DeltaPhi(AllMuons->at(a),*MET))));
                    //std::cout<< MT_LeadingMuon_MET <<std::endl;
                    //std::cout<< "Pass All  " ;
                    countX2Temp++;
                    if ( MT_LeadingMuon_MET > tempMT){
                        //TheMuon = AllMuons->at(a);
                        //std::cout<<i<<","<<a<<" = "<<AllMuons->at(a).Pt()<<std::endl;
                        TheMuon.SetPxPyPzE(AllMuons->at(a).px(), AllMuons->at(a).py(), AllMuons->at(a).pz(), AllMuons->at(a).energy());
                        tempMT = MT_LeadingMuon_MET;
                        countPass++;
                        Mupos = a;
                        MT_TheMuon_MET = MT_LeadingMuon_MET;
                        if (countX2Temp == 2 )countX2++;
                        if (countX2Temp > 2 )std::cout<<i<<": ERROR: More than the definition"<<std::endl;
                    }
                }//End if ISO
            }//End if ID
        }//End if pT
    }
    TheMuonB->Fill();
    TheMuonPossB->Fill();
    MT_TheMuon_METB->Fill();

    //std::cout<< "The Muon MT = " << tempMT<<std::endl;
    //std::cout<< "The Muon = " << TheMuon.Pt() <<std::endl;
    
}
std::cout<< "countAll = " << countAll <<std::endl;
std::cout<< "countPass = " << countPass <<std::endl;
std::cout<< "countX2 = " << countX2 <<std::endl;
std::cout<< "Diff(countAll,countPass) = " << -countAll+countPass <<std::endl;
Tree->Write(); 


countAll = 720628
countPass = 732562
countX2 = 11934
Diff(countAll,countPass) = 11934


In [10]:
f->Close();